In [1]:
import os
os.chdir("..")

In [2]:
import geopandas as gpd
from shapely import wkt
import polars as pl
import altair as alt
from src.data.data_process import DataClean
ds = DataClean()

In [ ]:
# df = ds.pull_dp05()
df.with_columns(
    under_5_year_male=pl.col("ratio") * pl.col("under_5_year") / (pl.col("ratio") + 100),
    pop_5_9_years_male=pl.col("ratio") * pl.col("pop_5_99") / (pl.col("ratio") + 100),
    pop_10_14_years_male=pl.col("ratio") * pl.col("pop_10_14_years") / (pl.col("ratio") + 100),
    pop_15_19_years_male=pl.col("ratio") * pl.col("pop_15_19_years") / (pl.col("ratio") + 100),
    pop_20_24_years_male=pl.col("ratio") * pl.col("pop_20_24_years") / (pl.col("ratio") + 100),
    pop_25_34_years_male=pl.col("ratio") * pl.col("pop_25_34_years") / (pl.col("ratio") + 100),
    pop_35_44_years_male=pl.col("ratio") * pl.col("pop_35_44_years") / (pl.col("ratio") + 100),
    pop_45_54_years_male=pl.col("ratio") * pl.col("pop_45_54_years") / (pl.col("ratio") + 100),
    pop_55_59_years_male=pl.col("ratio") * pl.col("pop_55_59_years") / (pl.col("ratio") + 100),
    pop_60_64_years_male=pl.col("ratio") * pl.col("pop_60_64_years") / (pl.col("ratio") + 100),
    pop_65_74_years_male=pl.col("ratio") * pl.col("pop_65_74_years") / (pl.col("ratio") + 100),
    pop_75_84_years_male=pl.col("ratio") * pl.col("pop_75_84_years") / (pl.col("ratio") + 100),
    over_85_years_male=pl.col("ratio") * pl.col("over_85_years") / (pl.col("ratio") + 100),
    under_5_year_female=100 * pl.col("under_5_year") / (pl.col("ratio") + 100),
    pop_5_9_years_female=100 * pl.col("pop_5_9_years") / (pl.col("ratio") + 100),
    pop_10_14_years_female=100 * pl.col("pop_10_14_years") / (pl.col("ratio") + 100),
    pop_15_19_years_female=100 * pl.col("pop_15_19_years") / (pl.col("ratio") + 100),
    pop_20_24_years_female=100 * pl.col("pop_20_24_years") / (pl.col("ratio") + 100),
    pop_25_34_years_female=100 * pl.col("pop_25_34_years") / (pl.col("ratio") + 100),
    pop_35_44_years_female=100 * pl.col("pop_35_44_years") / (pl.col("ratio") + 100),
    pop_45_54_years_female=100 * pl.col("pop_45_54_years") / (pl.col("ratio") + 100),
    pop_55_59_years_female=100 * pl.col("pop_55_59_years") / (pl.col("ratio") + 100),
    pop_60_64_years_female=100 * pl.col("pop_60_64_years") / (pl.col("ratio") + 100),
    pop_65_74_years_female=100 * pl.col("pop_65_74_years") / (pl.col("ratio") + 100),
    pop_75_84_years_female=100 * pl.col("pop_75_84_years") / (pl.col("ratio") + 100),
    over_85_years_female=100* pl.col("over_85_years") / (pl.col("ratio") + 100),
    )


year,geoid,total_pop,ratio,under_5_year,pop_5_9_years,pop_10_14_years,pop_15_19_years,pop_20_24_years,pop_25_34_years,pop_35_44_years,pop_45_54_years,pop_55_59_years,pop_60_64_years,pop_65_74_years,pop_75_84_years,over_85_years,under_5_year_male,pop_5_9_years_male,pop_10_14_years_male,pop_15_19_years_male,pop_20_24_years_male,pop_25_34_years_male,pop_35_44_years_male,pop_45_54_years_male,pop_55_59_years_male,pop_60_64_years_male,pop_65_74_years_male,pop_75_84_years_male,over_85_years_male,under_5_year_female,pop_5_9_years_female,pop_10_14_years_female,pop_15_19_years_female,pop_20_24_years_female,pop_25_34_years_female,pop_35_44_years_female,pop_45_54_years_female,pop_55_59_years_female,pop_60_64_years_female,pop_65_74_years_female,pop_75_84_years_female,over_85_years_female
i32,str,i32,f32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2012,"""7208317386""",511,24.0,6,11,30,57,50,24,143,85,22,11,48,0,50,1.16129,2.129032,5.806452,11.032258,9.677419,4.645161,27.677419,16.451613,4.258065,2.129032,9.290323,0.0,9.677419,4.83871,8.870968,24.193548,45.967742,40.322581,19.354839,115.322581,68.548387,17.741935,8.870968,38.709677,0.0,40.322581
2012,"""7208355957""",204,14.0,12,0,0,12,49,0,33,8,10,38,17,11,54,1.473684,0.0,0.0,1.473684,6.017544,0.0,4.052632,0.982456,1.22807,4.666667,2.087719,1.350877,6.631579,10.526316,0.0,0.0,10.526316,42.982456,0.0,28.947368,7.017544,8.77193,33.333333,14.912281,9.649123,47.368421
2012,"""7213983649""",537,33.0,83,74,60,29,57,75,15,23,17,42,0,29,21,20.593985,18.360902,14.887218,7.195489,14.142857,18.609023,3.721805,5.706767,4.218045,10.421053,0.0,7.195489,5.210526,62.406015,55.639098,45.112782,21.804511,42.857143,56.390977,11.278195,17.293233,12.781955,31.578947,0.0,21.804511,15.789474
2012,"""7202153162""",41309,2243.0,2286,2770,3122,3283,5410,4792,5826,2231,1764,4046,2783,753,39,2188.432778,2651.775501,2988.752027,3142.880495,5179.099445,4587.475886,5577.34443,2135.780196,1688.711908,3873.315408,2664.220657,720.861716,37.335467,97.567222,118.224499,133.247973,140.119505,230.900555,204.524114,248.65557,95.219804,75.288092,172.684592,118.779343,32.138284,1.664533
2012,"""7202134156""",40382,2111.0,1965,2343,2863,2971,5290,5003,5391,2786,2651,4071,2232,705,40,1876.126187,2237.029851,2733.511081,2836.626413,5050.741746,4776.722298,5147.173677,2659.993668,2531.099502,3886.87517,2131.050204,673.113976,38.190864,88.873813,105.970149,129.488919,134.373587,239.258254,226.277702,243.826323,126.006332,119.900498,184.12483,100.949796,31.886024,1.809136
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023,"""7215380166""",1050,98.900002,0,0,0,119,18,27,55,246,208,18,195,146,18,0.0,0.0,0.0,59.170943,8.950227,13.42534,27.347915,122.319764,103.424841,8.950227,96.960789,72.596283,8.950227,0.0,0.0,0.0,59.829062,9.049774,13.574661,27.652087,123.680245,104.575167,9.049774,98.039219,73.403723,9.049774
2023,"""7215381069""",3070,92.199997,22,186,110,306,170,419,299,346,153,268,512,255,24,10.55359,89.225805,52.767949,146.79084,81.550467,200.997915,143.43288,165.979185,73.39542,128.561912,245.610818,122.3257,11.513007,11.44641,96.774195,57.232051,159.20916,88.449533,218.002085,155.56712,180.020815,79.60458,139.438088,266.389182,132.6743,12.486993
2023,"""7215381155""",10538,84.900002,343,648,677,463,550,1393,1345,1255,756,558,1359,988,203,157.494329,297.540307,310.856154,212.594386,252.541927,639.619827,617.579804,576.254762,347.130358,256.215265,624.008144,453.657135,93.21093,185.505685,350.45972,366.143874,250.405633,297.458095,753.38023,727.420251,678.74529,408.869673,301.784758,734.991912,534.342906,109.789079


In [60]:
age_df = pl.read_parquet("data/external/age_cal.parquet")
age_df = age_df.with_columns(
    agg=pl.when((pl.col("Edad") < 5)).then(1)
          .when((pl.col("Edad") >= 5) & (pl.col("Edad") < 10)).then(2)
          .when((pl.col("Edad") >= 10) & (pl.col("Edad") < 15)).then(3)
          .when((pl.col("Edad") >= 15) & (pl.col("Edad") < 20)).then(4)
          .when((pl.col("Edad") >= 20) & (pl.col("Edad") < 25)).then(5)
          .when((pl.col("Edad") >= 25) & (pl.col("Edad") < 35)).then(6)
          .when((pl.col("Edad") >= 35) & (pl.col("Edad") < 45)).then(7)
          .when((pl.col("Edad") >= 45) & (pl.col("Edad") < 55)).then(8)
          .when((pl.col("Edad") >= 55) & (pl.col("Edad") < 60)).then(9)
          .when((pl.col("Edad") >= 60) & (pl.col("Edad") < 65)).then(10)
          .when((pl.col("Edad") >= 65) & (pl.col("Edad") < 75)).then(11)
          .when((pl.col("Edad") >= 75) & (pl.col("Edad") < 85)).then(12)
          .when((pl.col("Edad") >= 85)).then(13)
          .otherwise(-1)
    )

age_df = age_df.group_by("agg").agg(
    cal_male=pl.col("Caloriashombre").mean(),
    cal_female=pl.col("Caloriasmujer").mean()

)
age_df.sort("agg")

agg,cal_male,cal_female
i32,f64,f64
1,1184.1,1094.72
2,1644.35,1331.52
3,2104.6,1568.32
4,2550.982,1793.79
5,2654.56,1828.52
…,…,…
9,2362.31,1657.37
10,2320.56,1632.92
11,2257.935,1596.245


In [64]:
tmp = df
for col in df.select(pl.col("^pop_.*$")).columns:
    tmp = tmp.with_columns(
        (pl.col("ratio") * pl.col(col) / (pl.col("ratio") + 100)).alias(col + "_male"),
        (pl.col("ratio") * pl.col(col) / (pl.col("ratio") + 100)).alias(col + "_female"),
    )
tmp

year,geoid,total_pop,ratio,under_5_year,pop_5_9_years,pop_10_14_years,pop_15_19_years,pop_20_24_years,pop_25_34_years,pop_35_44_years,pop_45_54_years,pop_55_59_years,pop_60_64_years,pop_65_74_years,pop_75_84_years,over_85_years,pop_5_9_years_male,pop_5_9_years_female,pop_10_14_years_male,pop_10_14_years_female,pop_15_19_years_male,pop_15_19_years_female,pop_20_24_years_male,pop_20_24_years_female,pop_25_34_years_male,pop_25_34_years_female,pop_35_44_years_male,pop_35_44_years_female,pop_45_54_years_male,pop_45_54_years_female,pop_55_59_years_male,pop_55_59_years_female,pop_60_64_years_male,pop_60_64_years_female,pop_65_74_years_male,pop_65_74_years_female,pop_75_84_years_male,pop_75_84_years_female
i32,str,i32,f32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2012,"""7208317386""",511,24.0,6,11,30,57,50,24,143,85,22,11,48,0,50,2.129032,2.129032,5.806452,5.806452,11.032258,11.032258,9.677419,9.677419,4.645161,4.645161,27.677419,27.677419,16.451613,16.451613,4.258065,4.258065,2.129032,2.129032,9.290323,9.290323,0.0,0.0
2012,"""7208355957""",204,14.0,12,0,0,12,49,0,33,8,10,38,17,11,54,0.0,0.0,0.0,0.0,1.473684,1.473684,6.017544,6.017544,0.0,0.0,4.052632,4.052632,0.982456,0.982456,1.22807,1.22807,4.666667,4.666667,2.087719,2.087719,1.350877,1.350877
2012,"""7213983649""",537,33.0,83,74,60,29,57,75,15,23,17,42,0,29,21,18.360902,18.360902,14.887218,14.887218,7.195489,7.195489,14.142857,14.142857,18.609023,18.609023,3.721805,3.721805,5.706767,5.706767,4.218045,4.218045,10.421053,10.421053,0.0,0.0,7.195489,7.195489
2012,"""7202153162""",41309,2243.0,2286,2770,3122,3283,5410,4792,5826,2231,1764,4046,2783,753,39,2651.775501,2651.775501,2988.752027,2988.752027,3142.880495,3142.880495,5179.099445,5179.099445,4587.475886,4587.475886,5577.34443,5577.34443,2135.780196,2135.780196,1688.711908,1688.711908,3873.315408,3873.315408,2664.220657,2664.220657,720.861716,720.861716
2012,"""7202134156""",40382,2111.0,1965,2343,2863,2971,5290,5003,5391,2786,2651,4071,2232,705,40,2237.029851,2237.029851,2733.511081,2733.511081,2836.626413,2836.626413,5050.741746,5050.741746,4776.722298,4776.722298,5147.173677,5147.173677,2659.993668,2659.993668,2531.099502,2531.099502,3886.87517,3886.87517,2131.050204,2131.050204,673.113976,673.113976
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023,"""7215380166""",1050,98.900002,0,0,0,119,18,27,55,246,208,18,195,146,18,0.0,0.0,0.0,0.0,59.170943,59.170943,8.950227,8.950227,13.42534,13.42534,27.347915,27.347915,122.319764,122.319764,103.424841,103.424841,8.950227,8.950227,96.960789,96.960789,72.596283,72.596283
2023,"""7215381069""",3070,92.199997,22,186,110,306,170,419,299,346,153,268,512,255,24,89.225805,89.225805,52.767949,52.767949,146.79084,146.79084,81.550467,81.550467,200.997915,200.997915,143.43288,143.43288,165.979185,165.979185,73.39542,73.39542,128.561912,128.561912,245.610818,245.610818,122.3257,122.3257
2023,"""7215381155""",10538,84.900002,343,648,677,463,550,1393,1345,1255,756,558,1359,988,203,297.540307,297.540307,310.856154,310.856154,212.594386,212.594386,252.541927,252.541927,639.619827,639.619827,617.579804,617.579804,576.254762,576.254762,347.130358,347.130358,256.215265,256.215265,624.008144,624.008144,453.657135,453.657135
